In [42]:
import praw
import pandas as pd
import datetime

import xlsxwriter

import requests
import urllib
from tqdm import tqdm
import time
import os

In [43]:
# # SETTINGS
# download_pictures = False
# meme_number = 1000 # limited to 1000 by PRAW -> use pushshift API?

In [44]:
# def get_date(submission):
#     time = submission.created
#     return datetime.datetime.fromtimestamp(time)

In [45]:
# def save_image(url):
    
#     file_name = url.split("/")[-1]
#     res = requests.get(url, stream = True)

#     # Creating an images folder if it doens't already exist
#     if not os.path.exists('images'):
#        os.makedirs('images')

#     urllib.request.urlretrieve(url, "images/" + file_name)

In [100]:
reddit_read_only = praw.Reddit(
    client_id='JzbofoB1-Xrh3gbv--LVTQ',
    client_secret='v3R81WmJnSJJ8WWdjn4sZvRcerIVGg',
    user_agent='Ukraine Scraper'
)
# qwerrtty423499 username
# KVKb6AqeabAG2PD password
# cveodyiphyxqccujnc@tmmbt.com email
# v3R81WmJnSJJ8WWdjn4sZvRcerIVGg secret
meme_number = 1000
subreddit = reddit_read_only.subreddit("ukrainememes")

# posts = subreddit.hot(limit=10)
posts = subreddit.top(time_filter='year', limit=meme_number)
# Scraping the top posts of the current month



posts_dict = {"Title": [], "Author" : [], "Timestamp" : [], "Post Text": [],
            "ID": [], "Score": [],
            "Total Comments": [], "Image URL": []
            }
# t = tqdm(total=meme_number, bar_format='{l_bar}{bar:50}{r_bar}{bar:-1b}')
t = tqdm(total=meme_number, bar_format='{l_bar}{bar:50}{r_bar}{bar:-1b}')
with t as progress_bar:
    for post in posts:
        # URL of each post
        # ensures posts without images are ignored
        if (".jpg" in post.url or ".png" in post.url or ".tiff" in post.url or ".gif" in post.url):
            # image is valid
            posts_dict["Image URL"].append(str(post.url))
#             if (download_pictures):
#                 download_pictures(str(post.url))

        else:
            progress_bar.update()
            continue 

        # Title of each post
        posts_dict["Title"].append(post.title)

        # Author of each post
        posts_dict["Author"].append(str(post.author))

        # Time post was created
        posts_dict["Timestamp"].append(datetime.datetime.fromtimestamp(post.created).strftime("%m/%d/%Y, %H:%M:%S"))

        # Text inside a post
        posts_dict["Post Text"].append(post.selftext)

        # Unique ID of each post
        posts_dict["ID"].append(post.id)

        # The score of a post
        posts_dict["Score"].append(post.score)

        # Total number of comments inside the post
        posts_dict["Total Comments"].append(post.num_comments)
        
        progress_bar.update()


# Displaying the data in a pandas dataframe (for testing purposes)
pd.set_option('display.max_rows', 1000)
pd.set_option('max_colwidth', None)

top_posts = pd.DataFrame(posts_dict)
top_posts



 89%|████████████████████████████████████████████▍     | 888/1000 [00:11<00:01, 78.12it/s]                              


,Title,Author,Timestamp,Post Text,ID,Score,Total Comments,Image URL
0,rubles,silitbang6000,"03/09/2022, 11:39:52",,tabt2e,172,1,https://i.redd.it/vxpbivt3zdm81.png
1,You're Grounded,purplewhiteblack,"03/08/2022, 18:11:33",,t9tkjm,126,2,https://i.redd.it/jwc6d7r3s8m81.png
2,"'Z' is for Zelensky - The Times (London), Peter Brookes (3/9/2022)",None,"03/09/2022, 10:57:44",,taav8h,102,0,https://i.redd.it/rsrdgz9sqdm81.jpg
3,"Impact crater of Zelensky's balls, including visitors center, Cir: 2022",Tripledtities,"03/05/2022, 11:57:37",,t7dog3,101,0,https://i.redd.it/r11f2gjoill81.jpg
4,Are we NATO yet?,arcade_b1t,"10/04/2022, 04:05:56",,xv93ln,92,3,https://i.redd.it/gio0seawxqr91.png
5,Russian being rationed,arcade_b1t,"10/20/2022, 07:59:21",,y8wb9d,86,5,https://i.redd.it/7uqvj7u5ayu91.png
6,Reddit Moment,ShaddyDaddy123,"03/09/2022, 12:00:06",,tac9ad,82,1,https://i.redd.it/45yxtrps2em81.jpg
7,Bender bends tanks,Shadow336699,"03/07/2022, 18:47:37",,t933j5,81,0,https://i.redd.it/7rv7qu6ot1m81.png
8,For the wwII buffs.,BeHappyBeVegan,"10/13/2022, 07:27:31",,y2w8fa,79,21,https://i.redd.it/0qy5y4r56kt91.jpg
9,Tanks again,PyotrIvanov,"03/03/2022, 18:31:20",,t64top,77,1,https://i.redd.it/69za8cl479l81.jpg


In [72]:
# ------- MEMES FROM THE "ukrainememes" SUBREDDIT -------
def write_data(posts_dict, name):
    workbook = xlsxwriter.Workbook(name)
    worksheet = workbook.add_worksheet()

    # column, row

    # Writing the table headers
    i = 0
    for key in posts_dict:
        worksheet.write(0, i, key)
        i+=1

    # Writing the table data
    for x in range(1, len(posts_dict["Title"])):
        i = 0
        for key in posts_dict:
            if (key == "Image URL"):
                worksheet.write_url(x, i, posts_dict[key][x])
            else:
                worksheet.write(x, i, posts_dict[key][x])
            i+=1

    # Saving the document
    workbook.close()


In [73]:
# ------- MEMES FROM THE "ukrainememes" SUBREDDIT -------
def read_posts(post_list, posts_dict):
    for post in post_list:
        
        if (".jpg" in post["url"] or ".png" in post["url"] or ".tiff" in post["url"] or ".gif" in post["url"]):
            posts_dict["Image URL"].append(str(post["url"]))
#             if (download_pictures):
#                 download_pictures(str(post["url"]))
        else:
            continue 

        # Title of each post
        posts_dict["Title"].append(post["title"])

        # Author of each post
        posts_dict["Author"].append(str(post["author"]))

        # Time post was created
        posts_dict["Timestamp"].append(datetime.datetime.fromtimestamp(post["created_utc"]).strftime("%m/%d/%Y, %H:%M:%S"))

        # Text inside a post
        posts_dict["Post Text"].append(post["selftext"])

        # Unique ID of each post
        posts_dict["ID"].append(post["id"])

        # The score of a post
        posts_dict["Score"].append(post["score"])

        # Total number of comments inside the post
        posts_dict["Total Comments"].append(str(post["num_comments"]))
        
        
    

In [83]:
# ------- MEMES FROM THE "ukrainememes" SUBREDDIT -------

from pmaw import PushshiftAPI
# http://web.archive.org/web/20210123052853/https://pushshift.io/api-parameters/
# https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/

api = PushshiftAPI()
print(time.time())

# Settings
subreddit = "ukrainememes"
chunk_size = 100
reps = 3

posts_dict = {
    "Title": [], "Author" : [], "Timestamp" : [], "Post Text": [], 
    "ID": [], "Score": [], "Total Comments": [], "Image URL": []
}
    
# Iterating over the returned list of submissions
for x in range(reps):
    if (x == 0):
        last_time = int(time.time())
    else:
        if (len(post_list) != 0):
            last_time = post_list[-1]["created_utc"]
        else:
            break
    
    posts = api.search_submissions(
        subreddit=subreddit, 
        limit=chunk_size,
        before=last_time,
        sort="desc",
        sort_type="created_utc"
    )
    post_list = [post for post in posts]
    read_posts(post_list, posts_dict)

# Displaying the data in a pandas dataframe (for testing purposes)
pd.set_option('display.max_rows', 1000)
pd.set_option('max_colwidth', None)

top_posts = pd.DataFrame(posts_dict)
top_posts

# Writing the data to excel
write_data(posts_dict, "ukrainememes")



    

1673233555.4202278


In [87]:
# writes memes to "posts_dict"

def read_subreddit(subreddit, posts_dict, chunk_size, reps, use_keywords):

    api = PushshiftAPI()

    # Iterating over the returned list of submissions
    for x in range(reps):
        if (x == 0):
            last_time = int(time.time())
        else:
            if (len(post_list) != 0):
                last_time = post_list[-1]["created_utc"]
            else:
                break

        if (use_keywords):
            posts = api.search_submissions(
                subreddit=subreddit, 
                limit=chunk_size,
                before=last_time,
                sort="desc",
                q="russia|lugansk|luhansk|donetsk|ukraine|pushilin|kyiv|kiev|donbass|kharkiv|odesa|dnipro|zaporizhzhia|lviv|crimea|kryvyi|mykolaiv|sebastopol|sevastopol|mariupol|luhansk|vinnytsia|makiivka|simferopol|chernihiv|kherson|poltava|lutsk|oblast|dnipropetrovsk|zhitomir|rivne|sumy|ternopil|volin|zaporizhzhya|azov|zelensky|shmyhal|kuleba|reznikov|rada|pasechnik|yanukovych|poroshenko|yushchenko|dnieper|dniester|chernobyl|dpr|lpr|nato|dnr|lnr|maidan",
                sort_type="created_utc"
            )
        else:
            posts = api.search_submissions(
                subreddit=subreddit, 
                limit=chunk_size,
                before=last_time,
                sort="desc",
                sort_type="created_utc"
            )
            
        post_list = [post for post in posts]

        # Writing data to "posts_dict"
        read_posts(post_list, posts_dict)



    

In [91]:
# scrape memes from /memes that contain the keyword "ukraine" + unique
def get_memes(subreddits, file, keywords):

    chunk_size = 100
    reps = 3


    posts_dict = {
        "Title": [], "Author" : [], "Timestamp" : [], "Post Text": [], 
        "ID": [], "Score": [], "Total Comments": [], "Image URL": []
    }


    api = PushshiftAPI()


    for subreddit in subreddits:
        print(str(len(posts_dict["Title"])))
        if (len(keywords) != 0):
            read_subreddit(subreddit, posts_dict, chunk_size, reps, True)
        else:
            read_subreddit(subreddit, posts_dict, chunk_size, reps, False)

    write_data(posts_dict, file)


    # TODO: see if induvidual subreddits are being done correctly
        

In [93]:
# writes to "allreddits.xlsx"
key_subreddits = ["meme","memes","dankmemes","memes_of_the_dank","comedycemetary",
              "historymemes","terriblefacebookmemes","memesirl","therightcantmeme",
              "pewdiepiesubmissions","politicalcompassmemes"]

get_memes(key_subreddits, "allreddits.xlsx", [])

keyless_subreddits = ["ukraine22memes", "ukrainememes"]








TypeError: get_memes() takes 1 positional argument but 3 were given